<a href="https://colab.research.google.com/github/AlexMontgomerie/deepLearning/blob/master/denoise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Denoise Network

## Google Setup

In [0]:
#%cd /content
#!rm -rf deepLearning
#!git clone https://github.com/AlexMontgomerie/deepLearning
#%cd deepLearning
!pip install gputil
!pip install psutil
!pip install humanize
'''
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)
'''
# Taken from
# https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# Colab only provides one GPU and it is not always guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


## Setup

In [11]:
from common import *
!./setup.sh

--2019-02-25 19:48:08--  https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.box.com (imperialcollegelondon.box.com)... 107.152.26.197
Connecting to imperialcollegelondon.box.com (imperialcollegelondon.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-02-25 19:48:09--  https://imperialcollegelondon.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Reusing existing connection to imperialcollegelondon.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://imperialcollegelondon.app.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-02-25 19:48:09--  https://imperialcollegelondon.app.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.app.box.com (imperialcollegelondon.app.box.

## Get Denoise Model (Add)

In [12]:
from denoise_network import *
from get_data import get_data

EPOCHS = 50

seqs_train, seqs_test = get_data()

# get traning data
if os.path.exists('data/denoise_data/denoise_generator.npy') and os.path.exists('data/denoise_data/denoise_generator_val.npy'):
    denoise_generator     = np.load('data/denoise_data/denoise_generator.npy'    )
    denoise_generator_val = np.load('data/denoise_data/denoise_generator_val.npy')

else:
    denoise_generator     = DenoiseHPatches(seqs_train, batch_size=500)
    denoise_generator_val = DenoiseHPatches(seqs_test, batch_size=500)
    #np.save('data/denoise_data/denoise_generator.npy'    , denoise_generator    )
    #np.save('data/denoise_data/denoise_generator_val.npy', denoise_generator_val)


100%|██████████| 40/40 [00:56<00:00,  1.16s/it]


In [18]:

import sys
import json
import os
import glob
import keras
from keras import regularizers
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Input, UpSampling2D, concatenate
import time
import tensorflow as tf
import numpy as np
import cv2
import random
from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet
import matplotlib.pyplot as plt

#from common import *
#from denoise_network import *
#from get_data import get_data


# denoise model
def get_denoise_model(shape):

  inputs = Input(shape)
  conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
                 #kernel_regularizer=regularizers.l2(0.0001),activity_regularizer=regularizers.l1(0.00001),

  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  ## Bottleneck
  conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
                 #kernel_regularizer=regularizers.l2(0.0001), activity_regularizer=regularizers.l1(0.00001),


  ## Now the decoder starts
  up3 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv2))
               #kernel_regularizer=regularizers.l2(0.0001), activity_regularizer=regularizers.l1(0.00001),

  merge3 = concatenate([conv1,up3], axis = -1)
  conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge3)
                 #kernel_regularizer=regularizers.l2(0.0001), activity_regularizer=regularizers.l1(0.00001),


  conv4 = Conv2D(1, 3,  padding = 'same')(conv3)

  shallow_unet = Model(inputs = inputs, outputs = conv4)
  return shallow_unet

def get_denoise_model_add(shape):

  inputs = Input(shape)
  conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
                 #kernel_regularizer=regularizers.l2(0.0001),activity_regularizer=regularizers.l1(0.00001),

  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  ## Bottleneck
  conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
                 #kernel_regularizer=regularizers.l2(0.0001), activity_regularizer=regularizers.l1(0.00001),


  ## Now the decoder starts
  up3 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv2))
               #kernel_regularizer=regularizers.l2(0.0001), activity_regularizer=regularizers.l1(0.00001),

  merge3 = Add([conv1,up3], axis = -1)
  conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge3)
                 #kernel_regularizer=regularizers.l2(0.0001), activity_regularizer=regularizers.l1(0.00001),


  conv4 = Conv2D(1, 3,  padding = 'same')(conv3)

  shallow_unet = Model(inputs = inputs, outputs = conv4)
  return shallow_unet


# get model
shape = (32, 32, 1)
denoise_model = get_denoise_model(shape)

# callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 3, mode= 'auto'),
    keras.callbacks.ModelCheckpoint('data/denoise_model.weights.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
]

# optimiser
opt  = opt = keras.optimizers.nadam()

# loss
loss = 'mean_absolute_error'

# train network
denoise_model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
denoise_history = denoise_model.fit_generator(generator=denoise_generator, epochs=EPOCHS, callbacks=callbacks,
                                              verbose=1, validation_data=denoise_generator_val)

CUDNN_VERSION=7.4.2.24
_=/usr/bin/printenv
LANG=en_US.UTF-8
HOSTNAME=42e360808994
XRT_TPU_CONFIG=tpu_worker;0;10.101.254.34:8470
OLDPWD=/
CLOUDSDK_CONFIG=/content/.config
NVIDIA_VISIBLE_DEVICES=all
DATALAB_SETTINGS_OVERRIDES={"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=\"172.28.0.2\""]}
ENV=/root/.bashrc
PAGER=cat
NCCL_VERSION=2.4.2
TF_FORCE_GPU_ALLOW_GROWTH=true
JPY_PARENT_PID=41
NO_GCE_CHECK=True
PWD=/content/deepLearning
HOME=/root
CLICOLOR=1
DEBIAN_FRONTEND=noninteractive
COLAB_USE_SEABORN_STYLE=1
LIBRARY_PATH=/usr/local/cuda/lib64/stubs
GLIBCPP_FORCE_NEW=1
TERM=xterm-color
SHELL=/bin/bash
GCS_READ_CACHE_BLOCK_SIZE_MB=16
PYTHONWARNINGS=ignore:::pip._internal.cli.base_command
MPLBACKEND=module://ipykernel.pylab.backend_inline
CUDA_PKG_VERSION=10-0=10.0.130-1
CUDA_VERSION=10.0.130
NVIDIA_DRIVER_CAPABILITIES=compute,utility
COLAB_TPU_ADDR=10.101.254.34:8470
SHLVL=2
PYTHONPATH=/env/python
NVIDIA_REQUIRE_CUDA=cuda>=10.0 brand=tesla,driver>=38

KeyboardInterrupt: ignored